In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.tensor as T

import torchvision
import torchvision.transforms as transforms

import numpy as np

In [4]:
# import MNIST dataset
train = torchvision.datasets.MNIST(root='./mnist_data', train=True, transform=None, download=True)

Processing...
Done!


In [10]:
print(train.__len__)

<bound method MNIST.__len__ of <torchvision.datasets.mnist.MNIST object at 0x7faeecbead30>>


In [ ]:
# define network architecture

In [ ]:
# train network

In [ ]:
# evaluate performance